In [1]:
#librerias del proyecto
from sqlalchemy import create_engine
import pyodbc
import numpy as np
import re 
import pandas as pd 
import datetime

In [2]:
# conecion a base de datos
try:
    cnxn = pyodbc.connect('DSN=CARGA_BI;UID=xxxx;PWD=xxxx')
    cursor = cnxn.cursor()
    hora = datetime.datetime.now ()
    with open('Operacion_vt11.txt', 'a') as fobj:
        fobj.write(str(hora)+'conexion exitosa \n')
except BaseException as error:
    hora = datetime.datetime.now ()
    with open('Operacion_vt11.txt', 'a') as fobj:
        fobj.write(str(hora)+'conexion BBDD error: '+ str(error)+' \n' )     


In [3]:
vt11=pd.read_sql(""" select OPE_INDICADORES_PLANTA.transporte transporte,
          OPE_INDICADORES_PLANTA.[Creado por] Creado_por,
          OPE_INDICADORES_PLANTA.[Placa T] Placa_T,
          OPE_INDICADORES_PLANTA.Piloto Piloto,
          OPE_INDICADORES_PLANTA.Licencia Licencia,
          OPE_INDICADORES_PLANTA.inPlanCarg inPlanCarg,
          OPE_INDICADORES_PLANTA.horaplreg horaplreg,
          OPE_INDICADORES_PLANTA.HoraAcRg HoraAcRg,
          OPE_INDICADORES_PLANTA.HrPrIC HrPrIC,
          OPE_INDICADORES_PLANTA.HrAiCg HrAiCg,
          OPE_INDICADORES_PLANTA.Cump Cump0,
          OPE_INDICADORES_PLANTA.HrPrFiCga HrPrFiCga,
          OPE_INDICADORES_PLANTA.HrAFCg HrAFCg,
          OPE_INDICADORES_PLANTA.Cum Cum,
          OPE_INDICADORES_PLANTA.HoraPlDS HoraPlDS,
          OPE_INDICADORES_PLANTA.HrADpEXP HrADpEXP,
          OPE_INDICADORES_PLANTA.OTIF OTIF,
          OPE_INDICADORES_PLANTA.[Plan T Car] Plan_T_Car,
          OPE_INDICADORES_PLANTA.[Real T Car] Real_T_Car,
          OPE_INDICADORES_PLANTA.[Peso Teori] Peso_Teori,
          OPE_INDICADORES_PLANTA.[pESO rEAL] pESO_rEAL,
          OPE_INDICADORES_PLANTA.eNTREGAS eNTREGAS,
          OPE_INDICADORES_PLANTA.PTRP PTRP,
          OPE_INDICADORES_PLANTA.PLAn_TIEMPO_CARGA PLAn_TIEMPO_CARGA,
          OPE_INDICADORES_PLANTA.REAL_TIEMPO_CARGA REAL_TIEMPO_CARGA,
          OPE_INDICADORES_PLANTA.PLANTA PLANTA,
          OPE_INDICADORES_PLANTA.PAIS PAIS,
          OPE_INDICADORES_PLANTA.Cump Cump,
          OPE_INDICADORES_PLANTA.Cumple_tiempo_carga Cumple_tiempo_carga,
          OPE_INDICADORES_PLANTA.Cumple_inicio_carga Cumple_inicio_carga,
           OPE_INDICADORES_PLANTA.cumple_cliente cumple_cliente, 
    OPE_INDICADORES_PLANTA.Demora_tiempo_carga_R_C,DIF_PROM_INI_CARGA 
 ,  B.OTIF OBJETIVO_OTIF,
          B.CUMP_H_INI_CARGA  OBJETIVO_CUMP_H_INI_CARGA ,
          B.DEMORA_PROME_H_INI_CARGA OBJETIVO_DEMORA_PROME_H_INI_CARGA,
          B.CUMP_T_CARAGA OBJETIVO_CUMP_T_CARAGA,
          B.DEMORA_PROME_T_CARGA OBJETIVO_DEMORA_PROME_T_CARGA,
          B.CUMPLE_CLIENTE OBJETIVO_CUMPLE_CLIENTE
from   dbo.OPE_INDICADORES_PLANTA OPE_INDICADORES_PLANTA   LEFT JOIN    dbo.OPE_PLANTA_PLAN B 
ON (  OPE_INDICADORES_PLANTA.PTRP=B.PTRP)""", con=cnxn, parse_dates=True )

In [4]:
# limpieza de nulos
vt11['Creado_por'].fillna('', inplace=True)
vt11['Placa_T'].fillna('', inplace=True)
vt11['Piloto'].fillna('', inplace=True)
vt11['Licencia'].fillna('', inplace=True)
vt11['Creado_por']=vt11['Creado_por'].astype('str')
vt11['Placa_T']=vt11['Placa_T'].astype('str')
vt11['Piloto']=vt11['Piloto'].astype('str')
vt11['Licencia']=vt11['Licencia'].astype('str')

In [5]:
# Convert account_opened to datetime
vt11['inPlanCarg'] = pd.to_datetime(vt11['inPlanCarg'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce' ) 

In [6]:
#OTIF
VT11_OTIF=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['OTIF','OBJETIVO_OTIF']].mean()
VT11_OTIF=VT11_OTIF.reset_index()
VT11_OTIF['INDICADOR']='OTIF'
VT11_OTIF['UNIDAD']='%'
VT11_OTIF['OTIF']=VT11_OTIF['OTIF']*100
VT11_OTIF['OBJETIVO_OTIF']=VT11_OTIF['OBJETIVO_OTIF']*100
VT11_OTIF['STATUS']=VT11_OTIF['OTIF']/VT11_OTIF['OBJETIVO_OTIF']*100

In [12]:
#OTIF
# df['rainfall'].rolling(min_periods=7, window=7).sum()
VT11_OTIF_semana=VT11_OTIF
VT11_OTIF_semana['DIA_SEMANA']=VT11_OTIF_semana.inPlanCarg.dt.dayofweek
VT11_OTIF_semana['DIA_MES']=VT11_OTIF_semana.inPlanCarg.dt.day
VT11_OTIF_semana['ANIO']=VT11_OTIF_semana.inPlanCarg.dt.year
VT11_OTIF_semana['DIA_ANIO']=VT11_OTIF_semana.inPlanCarg.dt.dayofyear
VT11_OTIF_semana['NO_SEMANA']=VT11_OTIF_semana.inPlanCarg.dt.weekofyear
VT11_OTIF_semana['MES']=VT11_OTIF_semana.inPlanCarg.dt.month

In [8]:
#OTIF#semana
for  row_index,row  in VT11_OTIF_semana.iterrows():
    OTIF_SEMANAL=0
    v_planta=VT11_OTIF_semana['PLANTA']== VT11_OTIF_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_OTIF_semana['PAIS']==VT11_OTIF_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_OTIF_semana['NO_SEMANA']==VT11_OTIF_semana.loc[row_index ,'NO_SEMANA' ]
    v_Dia_semana=VT11_OTIF_semana['DIA_SEMANA']<=VT11_OTIF_semana.loc[row_index ,'DIA_SEMANA' ]
    OTIF_SEMANAL=VT11_OTIF_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['OTIF'].mean()
    VT11_OTIF_semana.loc[row_index, 'OTIF_SEMANAL']=OTIF_SEMANAL

In [10]:
#OTIF#Mes
for  row_index,row  in VT11_OTIF_semana.iterrows():
    OTIF_MENSUAL=0
    v_planta=VT11_OTIF_semana['PLANTA']== VT11_OTIF_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_OTIF_semana['PAIS']==VT11_OTIF_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_OTIF_semana['MES']==VT11_OTIF_semana.loc[row_index ,'MES' ]
    v_Dia_semana=VT11_OTIF_semana['DIA_MES']<=VT11_OTIF_semana.loc[row_index ,'DIA_MES' ]
    OTIF_MENSUAL=VT11_OTIF_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['OTIF'].mean()
    VT11_OTIF_semana.loc[row_index, 'OTIF_MENSUL']=OTIF_MENSUAL

In [16]:
#OTIF#anio
for  row_index,row  in VT11_OTIF_semana.iterrows():
    OTIF_ANIO=0
    v_planta=VT11_OTIF_semana['PLANTA']== VT11_OTIF_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_OTIF_semana['PAIS']==VT11_OTIF_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_OTIF_semana['ANIO']==VT11_OTIF_semana.loc[row_index ,'ANIO' ]
    v_Dia_semana=VT11_OTIF_semana['DIA_ANIO']<=VT11_OTIF_semana.loc[row_index ,'DIA_ANIO' ]
    OTIF_ANIO=VT11_OTIF_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['OTIF'].mean()
    VT11_OTIF_semana.loc[row_index, 'OTIF_ANIO']=OTIF_ANIO

In [41]:
VT11_OTIF_semana['STATUS_SEMANAL']=VT11_OTIF['OTIF_SEMANAL']/VT11_OTIF['OBJETIVO_OTIF']*100
VT11_OTIF_semana['STATUS_MENSUAL']=VT11_OTIF['OTIF_MENSUL']/VT11_OTIF['OBJETIVO_OTIF']*100
VT11_OTIF_semana['STATUS_ANUAL']=VT11_OTIF['OTIF_ANIO']/VT11_OTIF['OBJETIVO_OTIF']*100
VT11_OTIF_semana.fillna(0, inplace=True)    

In [23]:
# Convert account_opened to datetime
vt11['HrAiCg'] = pd.to_datetime(vt11['HrAiCg'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce' ) 
# Convert account_opened to datetime
vt11['HrPrFiCga'] = pd.to_datetime(vt11['HrPrFiCga'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce' ) 
vt11['Cump0']=vt11['Cump0'].astype('int')
vt11[['Cump0', 'HrAiCg','HrPrFiCga']]
vt11_DIFERENCIA_CUMPLE_CLIENTE=vt11['HrPrFiCga'] -  vt11['HrAiCg'] 
vt11["DIFERENCIA_CUMPLE_CLIENTE"]=abs(vt11_DIFERENCIA_CUMPLE_CLIENTE.dt.total_seconds())
for x,y in enumerate(vt11['DIFERENCIA_CUMPLE_CLIENTE']):
    if vt11.iloc[x,10]==1: 
        vt11.iloc[x,39]=np.nan

In [30]:
#Demora en el inicio de carga --OBJETIVO_DEMORA_PROME_H_INI_CARGA
VT11_DEMORA_INICIO_CLIENTE=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['DIFERENCIA_CUMPLE_CLIENTE','OBJETIVO_DEMORA_PROME_H_INI_CARGA']].mean()
VT11_DEMORA_INICIO_CLIENTE=VT11_DEMORA_INICIO_CLIENTE.reset_index()
VT11_DEMORA_INICIO_CLIENTE['INDICADOR']='Demora en el inicio de carga'
VT11_DEMORA_INICIO_CLIENTE['UNIDAD']='Minutos'
VT11_DEMORA_INICIO_CLIENTE['DIFERENCIA_CUMPLE_CLIENTE']=VT11_DEMORA_INICIO_CLIENTE['DIFERENCIA_CUMPLE_CLIENTE']/60

In [31]:
# df['rainfall'].rolling(min_periods=7, window=7).sum()
VT11_DEMORA_INICIO_CLIENTE_semana=VT11_DEMORA_INICIO_CLIENTE
VT11_DEMORA_INICIO_CLIENTE_semana['DIA_SEMANA']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.dayofweek
VT11_DEMORA_INICIO_CLIENTE_semana['DIA_MES']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.day
VT11_DEMORA_INICIO_CLIENTE_semana['ANIO']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.year
VT11_DEMORA_INICIO_CLIENTE_semana['DIA_ANIO']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.dayofyear
VT11_DEMORA_INICIO_CLIENTE_semana['NO_SEMANA']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.weekofyear
VT11_DEMORA_INICIO_CLIENTE_semana['MES']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.month

In [32]:
for  row_index,row  in VT11_DEMORA_INICIO_CLIENTE_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_DEMORA_INICIO_CLIENTE_semana['PLANTA']== VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_DEMORA_INICIO_CLIENTE_semana['PAIS']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_DEMORA_INICIO_CLIENTE_semana['NO_SEMANA']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'NO_SEMANA' ]
    v_Dia_semana=VT11_DEMORA_INICIO_CLIENTE_semana['DIA_SEMANA']<=VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'DIA_SEMANA' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_DEMORA_INICIO_CLIENTE_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['DIFERENCIA_CUMPLE_CLIENTE'].mean()
    VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index, 'DIFERENCIA_CUMPLE_CLIENTE_SEMANAL']=DEMORA_INICIO_CLIENTE_SEMANAL

In [34]:
for  row_index,row  in VT11_DEMORA_INICIO_CLIENTE_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_DEMORA_INICIO_CLIENTE_semana['PLANTA']== VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_DEMORA_INICIO_CLIENTE_semana['PAIS']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_DEMORA_INICIO_CLIENTE_semana['MES']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'MES' ]
    v_Dia_semana=VT11_DEMORA_INICIO_CLIENTE_semana['DIA_MES']<=VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'DIA_MES' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_DEMORA_INICIO_CLIENTE_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['DIFERENCIA_CUMPLE_CLIENTE'].mean()
    VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index, 'DIFERENCIA_CUMPLE_CLIENTE_MENSUAL']=DEMORA_INICIO_CLIENTE_SEMANAL

In [36]:
for  row_index,row  in VT11_DEMORA_INICIO_CLIENTE_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_DEMORA_INICIO_CLIENTE_semana['PLANTA']== VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_DEMORA_INICIO_CLIENTE_semana['PAIS']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_DEMORA_INICIO_CLIENTE_semana['ANIO']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'ANIO' ]
    v_Dia_semana=VT11_DEMORA_INICIO_CLIENTE_semana['DIA_ANIO']<=VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'DIA_ANIO' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_DEMORA_INICIO_CLIENTE_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['DIFERENCIA_CUMPLE_CLIENTE'].mean()
    VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index, 'DIFERENCIA_CUMPLE_CLIENTE_ANUAL']=DEMORA_INICIO_CLIENTE_SEMANAL
VT11_DEMORA_INICIO_CLIENTE_semana.fillna(0, inplace=True)    

In [42]:
VT11_DEMORA_INICIO_CLIENTE_semana

,inPlanCarg,PLANTA,PAIS,DIFERENCIA_CUMPLE_CLIENTE,OBJETIVO_DEMORA_PROME_H_INI_CARGA,INDICADOR,UNIDAD,DIA_SEMANA,DIA_MES,ANIO,DIA_ANIO,NO_SEMANA,MES,DIFERENCIA_CUMPLE_CLIENTE_SEMANAL,DIFERENCIA_CUMPLE_CLIENTE_MENSUAL,DIFERENCIA_CUMPLE_CLIENTE_ANUAL
0,2020-01-02,Bufalo,Costa Rica,0.000000,120.0,Demora en el inicio de carga,Minutos,3,2,2020,2,1,1,0.000000,0.000000,0.000000
1,2020-01-02,Orotina,Costa Rica,111.304348,120.0,Demora en el inicio de carga,Minutos,3,2,2020,2,1,1,111.304348,111.304348,111.304348
2,2020-01-02,Puerto Barrios,Guatemala,183.750000,0.0,Demora en el inicio de carga,Minutos,3,2,2020,2,1,1,183.750000,183.750000,183.750000
3,2020-01-02,Puerto Corinto,Nicaragua,22.789583,30.0,Demora en el inicio de carga,Minutos,3,2,2020,2,1,1,22.789583,22.789583,22.789583
4,2020-01-02,Puerto Quetzal,Guatemala,0.000000,0.0,Demora en el inicio de carga,Minutos,3,2,2020,2,1,1,0.000000,0.000000,0.000000
5,2020-01-02,Transporte Soledad,Colombia,0.000000,0.0,Demora en el inicio de carga,Minutos,3,2,2020,2,1,1,0.000000,0.000000,0.000000
6,2020-01-02,Transporte Yotoco,Colombia,105.000000,0.0,Demora en el inicio de carga,Minutos,3,2,2020,2,1,1,105.000000,105.000000,105.000000
7,2020-01-03,Orotina,Costa Rica,176.333333,120.0,Demora en el inicio de carga,Minutos,4,3,2020,3,1,1,143.818841,143.818841,143.818841
8,2020-01-03,Puerto Barrios,Guatemala,175.000000,0.0,Demora en el inicio de carga,Minutos,4,3,2020,3,1,1,179.375000,179.375000,179.375000
9,2020-01-03,Puerto Corinto,Nicaragua,25.794444,30.0,Demora en el inicio de carga,Minutos,4,3,2020,3,1,1,24.292014,24.292014,24.292014


In [44]:
 #Cumple_inicio_carga
VT11_CUMPLE_INI_CARGA=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['Cumple_inicio_carga','OBJETIVO_CUMP_H_INI_CARGA']].mean()
VT11_CUMPLE_INI_CARGA=VT11_CUMPLE_INI_CARGA.reset_index()
VT11_CUMPLE_INI_CARGA['INDICADOR']='Cumplimiento Inicio de Carga'
VT11_CUMPLE_INI_CARGA['UNIDAD']='%'
VT11_CUMPLE_INI_CARGA['Cumple_inicio_carga']=VT11_CUMPLE_INI_CARGA['Cumple_inicio_carga']*100

In [45]:
 #Cumple_inicio_carga
# df['rainfall'].rolling(min_periods=7, window=7).sum()
VT11_CUMPLE_INI_CARGA_semana=VT11_CUMPLE_INI_CARGA
VT11_CUMPLE_INI_CARGA_semana['DIA_SEMANA']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.dayofweek
VT11_CUMPLE_INI_CARGA_semana['DIA_MES']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.day
VT11_CUMPLE_INI_CARGA_semana['ANIO']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.year
VT11_CUMPLE_INI_CARGA_semana['DIA_ANIO']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.dayofyear
VT11_CUMPLE_INI_CARGA_semana['NO_SEMANA']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.weekofyear
VT11_CUMPLE_INI_CARGA_semana['MES']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.month

In [50]:
 #Cumple_inicio_carga
for  row_index,row  in VT11_CUMPLE_INI_CARGA_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_CUMPLE_INI_CARGA_semana['PLANTA']== VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_CUMPLE_INI_CARGA_semana['PAIS']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_CUMPLE_INI_CARGA_semana['NO_SEMANA']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'NO_SEMANA' ]
    v_Dia_semana=VT11_CUMPLE_INI_CARGA_semana['DIA_SEMANA']<=VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'DIA_SEMANA' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_CUMPLE_INI_CARGA_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Cumple_inicio_carga'].mean()
    VT11_CUMPLE_INI_CARGA_semana.loc[row_index, 'Cumplimiento Inicio de Carga_SEMANAL']=DEMORA_INICIO_CLIENTE_SEMANAL

In [49]:
 #Cumple_inicio_carga
for  row_index,row  in VT11_CUMPLE_INI_CARGA_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_CUMPLE_INI_CARGA_semana['PLANTA']== VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_CUMPLE_INI_CARGA_semana['PAIS']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_CUMPLE_INI_CARGA_semana['MES']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'MES' ]
    v_Dia_semana=VT11_CUMPLE_INI_CARGA_semana['DIA_MES']<=VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'DIA_MES' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_CUMPLE_INI_CARGA_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Cumple_inicio_carga'].mean()
    VT11_CUMPLE_INI_CARGA_semana.loc[row_index, 'Cumplimiento Inicio de Carga_MENSUAL']=DEMORA_INICIO_CLIENTE_SEMANAL

In [51]:
 #Cumple_inicio_carga
for  row_index,row  in VT11_CUMPLE_INI_CARGA_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_CUMPLE_INI_CARGA_semana['PLANTA']== VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_CUMPLE_INI_CARGA_semana['PAIS']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_CUMPLE_INI_CARGA_semana['ANIO']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'ANIO' ]
    v_Dia_semana=VT11_CUMPLE_INI_CARGA_semana['DIA_ANIO']<=VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'DIA_ANIO' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_CUMPLE_INI_CARGA_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Cumple_inicio_carga'].mean()
    VT11_CUMPLE_INI_CARGA_semana.loc[row_index, 'Cumplimiento Inicio de Carga_ANUAL']=DEMORA_INICIO_CLIENTE_SEMANAL
VT11_DEMORA_INICIO_CLIENTE_semana.fillna(0, inplace=True)    

In [66]:
 #Cumple_inicio_carga
VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']=VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']*100
VT11_CUMPLE_INI_CARGA_semana['STATUS']=VT11_CUMPLE_INI_CARGA_semana['Cumple_inicio_carga']/VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']*100
VT11_CUMPLE_INI_CARGA_semana['STATUS_SEMANAL']=VT11_CUMPLE_INI_CARGA_semana['Cumplimiento Inicio de Carga_SEMANAL']/VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']*100
VT11_CUMPLE_INI_CARGA_semana['STATUS_MENSUAL']=VT11_CUMPLE_INI_CARGA_semana['Cumplimiento Inicio de Carga_MENSUAL']/VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']*100
VT11_CUMPLE_INI_CARGA_semana['STATUS_ANUAL']=VT11_CUMPLE_INI_CARGA_semana['Cumplimiento Inicio de Carga_ANUAL']/VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']*100
VT11_CUMPLE_INI_CARGA_semana.fillna(0, inplace=True)    

In [68]:
 #Cumple_inicio_carga
VT11_CUMPLE_INI_CARGA_semana

,inPlanCarg,PLANTA,PAIS,Cumple_inicio_carga,OBJETIVO_CUMP_H_INI_CARGA,INDICADOR,UNIDAD,DIA_SEMANA,DIA_MES,ANIO,DIA_ANIO,NO_SEMANA,MES,Cumplimiento Inicio de Carga_MENSUAL,Cumplimiento Inicio de Carga_SEMANAL,Cumplimiento Inicio de Carga_ANUAL,STATUS,STATUS_SEMANAL,STATUS_MENSUAL,STATUS_ANUAL
0,2020-01-02,Bufalo,Costa Rica,0.000000,75.000000,Cumplimiento Inicio de Carga,%,3,2,2020,2,1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-01-02,Orotina,Costa Rica,23.333333,85.000002,Cumplimiento Inicio de Carga,%,3,2,2020,2,1,1,23.333333,23.333333,23.333333,27.450980,27.450980,27.450980,27.450980
2,2020-01-02,Puerto Barrios,Guatemala,55.555556,0.000000,Cumplimiento Inicio de Carga,%,3,2,2020,2,1,1,55.555556,55.555556,55.555556,inf,inf,inf,inf
3,2020-01-02,Puerto Corinto,Nicaragua,20.000000,89.999998,Cumplimiento Inicio de Carga,%,3,2,2020,2,1,1,20.000000,20.000000,20.000000,22.222223,22.222223,22.222223,22.222223
4,2020-01-02,Puerto Quetzal,Guatemala,100.000000,0.000000,Cumplimiento Inicio de Carga,%,3,2,2020,2,1,1,100.000000,100.000000,100.000000,inf,inf,inf,inf
5,2020-01-02,Transporte Soledad,Colombia,100.000000,0.000000,Cumplimiento Inicio de Carga,%,3,2,2020,2,1,1,100.000000,100.000000,100.000000,inf,inf,inf,inf
6,2020-01-02,Transporte Yotoco,Colombia,91.666667,0.000000,Cumplimiento Inicio de Carga,%,3,2,2020,2,1,1,91.666667,91.666667,91.666667,inf,inf,inf,inf
7,2020-01-03,Orotina,Costa Rica,58.620690,85.000002,Cumplimiento Inicio de Carga,%,4,3,2020,3,1,1,40.977011,40.977011,40.977011,68.965515,48.208247,48.208247,48.208247
8,2020-01-03,Puerto Barrios,Guatemala,87.500000,0.000000,Cumplimiento Inicio de Carga,%,4,3,2020,3,1,1,71.527778,71.527778,71.527778,inf,inf,inf,inf
9,2020-01-03,Puerto Corinto,Nicaragua,81.250000,89.999998,Cumplimiento Inicio de Carga,%,4,3,2020,3,1,1,50.625000,50.625000,50.625000,90.277780,56.250001,56.250001,56.250001


In [117]:
#cumple_cliente
VT11_CUMPLE_CLIENTE=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['cumple_cliente']].mean()
VT11_CUMPLE_CLIENTE=VT11_CUMPLE_CLIENTE.reset_index()
VT11_CUMPLE_CLIENTE['INDICADOR']='Cumplimiento cliente'
VT11_CUMPLE_CLIENTE['UNIDAD']='%'
VT11_CUMPLE_CLIENTE['cumple_cliente']=VT11_CUMPLE_CLIENTE['cumple_cliente']*100

In [119]:
 #Cumple_tiempo_carga
VT11_CUMPLE_TIEMPO_CARGA=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['Cumple_tiempo_carga']].mean()
VT11_CUMPLE_TIEMPO_CARGA=VT11_CUMPLE_TIEMPO_CARGA.reset_index()
VT11_CUMPLE_TIEMPO_CARGA['INDICADOR']='Cumplimiento Tiempo de Carga'
VT11_CUMPLE_TIEMPO_CARGA['UNIDAD']='%'
VT11_CUMPLE_TIEMPO_CARGA['Cumple_tiempo_carga']=VT11_CUMPLE_TIEMPO_CARGA['Cumple_tiempo_carga']*100

In [131]:
#Total de toneladas métricas producidas
VT11_TONELADAS_PRODUCIDAS=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['pESO_rEAL']].sum()
VT11_TONELADAS_PRODUCIDAS=VT11_TONELADAS_PRODUCIDAS.reset_index()
VT11_TONELADAS_PRODUCIDAS['INDICADOR']='Total de toneladas métricas producidas'
VT11_TONELADAS_PRODUCIDAS['UNIDAD']='Ton'
VT11_TONELADAS_PRODUCIDAS['pESO_rEAL']=VT11_TONELADAS_PRODUCIDAS['pESO_rEAL']/1000

In [133]:
#Total Camiones Atendido
VT11_CAMIONES_ATENDIDOS=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['transporte']].count()
VT11_CAMIONES_ATENDIDOS=VT11_CAMIONES_ATENDIDOS.reset_index()
VT11_CAMIONES_ATENDIDOS['INDICADOR']='Total Camiones Atendidos'
VT11_CAMIONES_ATENDIDOS['UNIDAD']=''


In [136]:
# Convert account_opened to datetime
vt11['Plan_T_Car'] = pd.to_datetime(vt11['Plan_T_Car'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce' ) 
# Convert account_opened to datetime
vt11['Real_T_Car'] = pd.to_datetime(vt11['Real_T_Car'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce' ) 

In [153]:
vt11_Demora_carga=vt11['Plan_T_Car'] -  vt11['Real_T_Car'] 
vt11["Demora_tiempo_carga"]=abs(vt11_Demora_carga.dt.total_seconds())

In [154]:
#          OPE_INDICADORES_PLANTA.[Plan T Car] Plan_T_Car,
#          OPE_INDICADORES_PLANTA.[Real T Car] Real_T_Car,
# vt11.iloc[: ,[-1]]

for x,y in enumerate(vt11['Plan_T_Car']):
    if vt11.iloc[x,17]>vt11.iloc[x,18]:
        vt11.iloc[x,-1]=np.nan


In [155]:
#DIFERENCIA_CUMPLE_CLIENTE
VT11_DEMORA_TIEMPO_CARGA=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['Demora_tiempo_carga']].sum()
VT11_DEMORA_TIEMPO_CARGA=VT11_DEMORA_TIEMPO_CARGA.reset_index()
VT11_DEMORA_TIEMPO_CARGA['INDICADOR']='Demora en el tiempo de carga'
VT11_DEMORA_TIEMPO_CARGA['UNIDAD']='Minutos'
VT11_DEMORA_TIEMPO_CARGA['Demora_tiempo_carga']=VT11_DEMORA_TIEMPO_CARGA['Demora_tiempo_carga']/60

In [157]:
VT11_OTIF
VT11_CUMPLE_CLIENTE
VT11_CUMPLE_INI_CARGA
VT11_DEMORA_INICIO_CLIENTE
VT11_CUMPLE_TIEMPO_CARGA
VT11_TONELADAS_PRODUCIDAS
VT11_DEMORA_TIEMPO_CARGA
VT11_CAMIONES_ATENDIDOS

,inPlanCarg,PLANTA,PAIS,transporte,INDICADOR,UNIDAD
0,2020-01-02,Bufalo,Costa Rica,4,Total Camiones Atendidos,
1,2020-01-02,Orotina,Costa Rica,30,Total Camiones Atendidos,
2,2020-01-02,Puerto Barrios,Guatemala,9,Total Camiones Atendidos,
3,2020-01-02,Puerto Corinto,Nicaragua,10,Total Camiones Atendidos,
4,2020-01-02,Puerto Quetzal,Guatemala,6,Total Camiones Atendidos,
5,2020-01-02,Transporte Soledad,Colombia,4,Total Camiones Atendidos,
6,2020-01-02,Transporte Yotoco,Colombia,12,Total Camiones Atendidos,
7,2020-01-03,Orotina,Costa Rica,29,Total Camiones Atendidos,
8,2020-01-03,Puerto Barrios,Guatemala,8,Total Camiones Atendidos,
9,2020-01-03,Puerto Corinto,Nicaragua,16,Total Camiones Atendidos,
